In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the data.csv.
import pandas as pd 
data_df = pd.read_csv("archive/data.csv")
data_df.head()

,Unnamed: 0,Unnamed: 0.1,ID,Name,Sex,Age,Height,Weight,Team,NOC,...,Event,Medal,Host,Nat_Pop,Global_Pop%,Rural_Pop%,Life_Expect,GDP,GDP_Per_Cap,Global_GDP%
0,0,0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,...,Basketball Men's Basketball,NaN,0,1.164970e+09,0.213515,71.800,69.587000,4.270000e+11,366.5330438,0.016811
1,1,1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,...,Judo Men's Extra-Lightweight,NaN,0,1.350695e+09,0.190308,48.111,75.602000,8.560000e+12,6337.478113,0.114286
2,2,2,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NLD,...,Speed Skating Women's 500 metres,NaN,0,1.476009e+07,0.002891,32.094,76.890244,2.590000e+11,17547.31372,0.013560
3,3,3,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NLD,...,"Speed Skating Women's 1,000 metres",NaN,0,1.476009e+07,0.002891,32.094,76.890244,2.590000e+11,17547.31372,0.013560
4,4,4,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NLD,...,Speed Skating Women's 500 metres,NaN,0,1.518417e+07,0.002783,29.777,77.217073,3.580000e+11,23577.19219,0.014094


In [2]:
#Drop unnecessary columns
data_df = data_df.drop(["Unnamed: 0","Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Team", "Games","Year","City", "Nat_Pop", "GDP"], 1)

#data_df = data_df.drop(["GDP_Per_Cap"], 1)
data_df = data_df[data_df.GDP_Per_Cap != '#DIV/0!']

#Rename Global_Pop% and Rural_Pop% to make it easier to call in future
data_df.rename(columns = {'Global_Pop%':'Global_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Rural_Pop%':'Rural_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Global_GDP%':'Global_GDP_Perc'}, inplace = True)


data_df.head()

C:\Users\14043\AppData\Local\Temp/ipykernel_15680/3203423478.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Unnamed: 0","Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Team", "Games","Year","City", "Nat_Pop", "GDP"], 1)


,NOC,Season,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,CHN,Summer,Basketball,NaN,0,0.213515,71.800,69.587000,366.5330438,0.016811
1,CHN,Summer,Judo,NaN,0,0.190308,48.111,75.602000,6337.478113,0.114286
2,NLD,Winter,Speed Skating,NaN,0,0.002891,32.094,76.890244,17547.31372,0.013560
3,NLD,Winter,Speed Skating,NaN,0,0.002891,32.094,76.890244,17547.31372,0.013560
4,NLD,Winter,Speed Skating,NaN,0,0.002783,29.777,77.217073,23577.19219,0.014094


In [3]:
#Filter out season data to just summer
data_df = data_df[data_df["Season"].str.contains("Winter") == False]

#Filter out hockey data since it is a winter sport
data_df = data_df[data_df["Sport"].str.contains("Hockey") == False]

data_df.head()

,NOC,Season,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,CHN,Summer,Basketball,NaN,0,0.213515,71.800,69.587000,366.5330438,0.016811
1,CHN,Summer,Judo,NaN,0,0.190308,48.111,75.602000,6337.478113,0.114286
26,FIN,Summer,Badminton,NaN,0,0.000846,17.817,77.465854,24342.1392,0.003750
29,FIN,Summer,Athletics,NaN,0,0.000846,17.817,77.465854,24342.1392,0.003750
50,NOR,Summer,Handball,Gold,0,0.000705,21.474,80.592683,97101.38727,0.007291


In [4]:
#Turn Medal Values into Boolean
data_df.fillna(value = False,
          inplace = True)

#Drop Season since no longer needed after filtering
data_df = data_df.drop(["Season"],1)
data_df.head()

C:\Users\14043\AppData\Local\Temp/ipykernel_15680/516755359.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Season"],1)


,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,CHN,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811
1,CHN,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
50,NOR,Handball,Gold,0,0.000705,21.474,80.592683,97101.38727,0.007291


In [5]:
#Replace medal values to true to finish boolean
data_df = data_df.replace({"Gold" : True})
data_df = data_df.replace({"Silver" : True})
data_df = data_df.replace({"Bronze" : True})
data_df.head(15)


,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,CHN,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811
1,CHN,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
50,NOR,Handball,True,0,0.000705,21.474,80.592683,97101.38727,0.007291
51,ROU,Weightlifting,False,0,0.002648,45.251,75.012195,9540.579969,0.00248
52,NOR,Wrestling,False,0,0.000734,23.919,78.634146,38076.43209,0.005089
53,NOR,Wrestling,False,0,0.000713,22.725,79.841463,57492.41601,0.006027
58,NLD,Water Polo,False,0,0.003030,33.681,76.233171,9844.559262,0.011736
60,NLD,Rowing,True,0,0.002681,26.36,77.43561,28717.68825,0.014114


In [6]:
#Get Sport count to use for later dataframe creation
Sport_counts = data_df["Sport"].value_counts()
Sport_counts

Athletics                27915
Swimming                 19902
Gymnastics               18095
Cycling                   7886
Rowing                    7800
Shooting                  7274
Fencing                   6309
Canoeing                  5529
Wrestling                 5170
Football                  4831
Boxing                    4648
Equestrianism             4474
Judo                      3774
Basketball                3563
Handball                  3535
Volleyball                3384
Weightlifting             3145
Water Polo                2746
Archery                   2057
Diving                    2051
Tennis                    2016
Table Tennis              1937
Badminton                 1455
Modern Pentathlon         1254
Synchronized Swimming      895
Baseball                   894
Rhythmic Gymnastics        654
Taekwondo                  606
Beach Volleyball           564
Triathlon                  529
Softball                   478
Rugby Sevens               299
Trampoli

In [7]:
#Begin binning data to make it easier to get dummies
#Find min and max of data types to create bins
data_df['Global_Pop_Perc'] = data_df['Global_Pop_Perc'].astype(float, errors = 'raise')

min_value = data_df['Global_Pop_Perc'].min()
max_value = data_df['Global_Pop_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

names = ["0","1","2","3","4","5","6","7","8"]
print(bins)
print (names)


1.45e-06
0.224549844
[1.45000000e-06 2.49512716e-02 4.99010931e-02 7.48509147e-02
 9.98007362e-02 1.24750558e-01 1.49700379e-01 1.74650201e-01
 1.99600022e-01 2.24549844e-01]
['0', '1', '2', '3', '4', '5', '6', '7', '8']


In [8]:
data_df["Global_Pop_Bin"] = pd.cut(data_df["Global_Pop_Perc"], bins, labels=names)
data_df

,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc,Global_Pop_Bin
0,CHN,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811,8
1,CHN,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286,7
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375,0
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375,0
50,NOR,Handball,True,0,0.000705,21.474,80.592683,97101.38727,0.007291,0
...,...,...,...,...,...,...,...,...,...,...
204020,RUS,Athletics,True,0,0.022383,26.582,65.470976,4102.25644,0.013493,0
204024,USA,Football,False,0,0.054676,26.377,71.156098,6098.258185,0.339523,2
204025,RUS,Rowing,False,0,0.023961,26.65,65.483659,1773.575078,0.007738,0
204026,RUS,Rowing,False,0,0.022383,26.582,65.470976,4102.25644,0.013493,0


In [9]:
data_df['GDP_Per_Cap'] = data_df['GDP_Per_Cap'].astype(float, errors = 'raise')

min_value = data_df['GDP_Per_Cap'].min()
max_value = data_df['GDP_Per_Cap'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

0.0
192989.1895


In [10]:
data_df["GDP_Per_Cap"] = pd.cut(data_df["GDP_Per_Cap"], bins, labels=names)
data_df

,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc,Global_Pop_Bin
0,CHN,Basketball,False,0,0.213515,71.8,69.587,0,0.016811,8
1,CHN,Judo,False,0,0.190308,48.111,75.602,0,0.114286,7
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,1,0.00375,0
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,1,0.00375,0
50,NOR,Handball,True,0,0.000705,21.474,80.592683,4,0.007291,0
...,...,...,...,...,...,...,...,...,...,...
204020,RUS,Athletics,True,0,0.022383,26.582,65.470976,0,0.013493,0
204024,USA,Football,False,0,0.054676,26.377,71.156098,0,0.339523,2
204025,RUS,Rowing,False,0,0.023961,26.65,65.483659,0,0.007738,0
204026,RUS,Rowing,False,0,0.022383,26.582,65.470976,0,0.013493,0


In [11]:
data_df['Rural_Pop_Perc'] = data_df['Rural_Pop_Perc'].astype(float, errors = 'raise')

min_value = data_df['Rural_Pop_Perc'].min()
max_value = data_df['Rural_Pop_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

0.0
96.305


In [12]:
data_df["Rural_Pop_Bin"] = pd.cut(data_df["Rural_Pop_Perc"], bins, labels=names)
data_df

,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc,Global_Pop_Bin,Rural_Pop_Bin
0,CHN,Basketball,False,0,0.213515,71.800,69.587,0,0.016811,8,6
1,CHN,Judo,False,0,0.190308,48.111,75.602,0,0.114286,7,4
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,1,0.00375,0,1
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,1,0.00375,0,1
50,NOR,Handball,True,0,0.000705,21.474,80.592683,4,0.007291,0,2
...,...,...,...,...,...,...,...,...,...,...,...
204020,RUS,Athletics,True,0,0.022383,26.582,65.470976,0,0.013493,0,2
204024,USA,Football,False,0,0.054676,26.377,71.156098,0,0.339523,2,2
204025,RUS,Rowing,False,0,0.023961,26.650,65.483659,0,0.007738,0,2
204026,RUS,Rowing,False,0,0.022383,26.582,65.470976,0,0.013493,0,2


In [13]:
data_df['Life_Expect'] = data_df['Life_Expect'].astype(float, errors = 'raise')

min_value = data_df['Life_Expect'].min()
max_value = data_df['Life_Expect'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

data_df["Life_Expect_Bin"] = pd.cut(data_df["Life_Expect"], bins, labels=names)
data_df

0.0
85.41707317


,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin
0,CHN,Basketball,False,0,0.213515,71.800,69.587000,0,0.016811,8,6,7
1,CHN,Judo,False,0,0.190308,48.111,75.602000,0,0.114286,7,4,7
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,1,0.00375,0,1,8
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,1,0.00375,0,1,8
50,NOR,Handball,True,0,0.000705,21.474,80.592683,4,0.007291,0,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...
204020,RUS,Athletics,True,0,0.022383,26.582,65.470976,0,0.013493,0,2,6
204024,USA,Football,False,0,0.054676,26.377,71.156098,0,0.339523,2,2,7
204025,RUS,Rowing,False,0,0.023961,26.650,65.483659,0,0.007738,0,2,6
204026,RUS,Rowing,False,0,0.022383,26.582,65.470976,0,0.013493,0,2,6


In [14]:
data_df.dtypes

NOC                  object
Sport                object
Medal                  bool
Host                  int64
Global_Pop_Perc     float64
Rural_Pop_Perc      float64
Life_Expect         float64
GDP_Per_Cap        category
Global_GDP_Perc      object
Global_Pop_Bin     category
Rural_Pop_Bin      category
Life_Expect_Bin    category
dtype: object

In [15]:
min_value = data_df['Global_GDP_Perc'].min()
max_value = data_df['Global_GDP_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

names = ["0","1","2","3","4","5","6","7","8"]

data_df["Global_GDP_Perc_Bin"] = pd.cut(data_df["Global_GDP_Perc"], bins, labels=names)
data_df

False
0.3963503649635037


,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
0,CHN,Basketball,False,0,0.213515,71.800,69.587000,0,0.016811,8,6,7,0
1,CHN,Judo,False,0,0.190308,48.111,75.602000,0,0.114286,7,4,7,2
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,1,0.00375,0,1,8,0
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,1,0.00375,0,1,8,0
50,NOR,Handball,True,0,0.000705,21.474,80.592683,4,0.007291,0,2,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204020,RUS,Athletics,True,0,0.022383,26.582,65.470976,0,0.013493,0,2,6,0
204024,USA,Football,False,0,0.054676,26.377,71.156098,0,0.339523,2,2,7,7
204025,RUS,Rowing,False,0,0.023961,26.650,65.483659,0,0.007738,0,2,6,0
204026,RUS,Rowing,False,0,0.022383,26.582,65.470976,0,0.013493,0,2,6,0


In [16]:
data_df = data_df.drop(["Global_Pop_Perc", "Rural_Pop_Perc", "Life_Expect", "Global_GDP_Perc"], 1)
data_df.head()

C:\Users\14043\AppData\Local\Temp/ipykernel_15680/4282354263.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Global_Pop_Perc", "Rural_Pop_Perc", "Life_Expect", "Global_GDP_Perc"], 1)


,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
0,CHN,Basketball,False,0,0,8,6,7,0
1,CHN,Judo,False,0,0,7,4,7,2
26,FIN,Badminton,False,0,1,0,1,8,0
29,FIN,Athletics,False,0,1,0,1,8,0
50,NOR,Handball,True,0,4,0,2,8,0


Athletics Logistic Regression

In [17]:
# Creating on the basis of Athlethics
Athletics_df = data_df[data_df['Sport'] == "Athletics"]
Athletics_df = Athletics_df.dropna()
 
print(Athletics_df)

        NOC      Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
29      FIN  Athletics  False     0           1              0             1   
62      FRA  Athletics  False     0           1              0             1   
81      EST  Athletics  False     0           0              0             2   
82      EST  Athletics  False     0           0              0             2   
91      FRA  Athletics  False     0           0              0             2   
...     ...        ...    ...   ...         ...            ...           ...   
203977  KAZ  Athletics  False     0           0              0             4   
203978  KAZ  Athletics  False     0           0              0             4   
203979  KAZ  Athletics  False     0           0              0             4   
204019  RUS  Athletics   True     0           0              0             2   
204020  RUS  Athletics   True     0           0              0             2   

       Life_Expect_Bin Global_GDP_Perc_

In [18]:
Athletics_dummies = pd.get_dummies(Athletics_df)
Athletics_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AFG,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
29,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
62,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
81,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [19]:
from sklearn.model_selection import train_test_split

# Convert categorical data to numeric and separate target feature for training data
x_train = Athletics_dummies.drop(columns=['Medal'])
y_train = Athletics_dummies['Medal']

In [20]:
# Convert categorical data to numeric and separate target feature for testing data
x_test = Athletics_dummies.drop(columns=['Medal'])
y_test = Athletics_dummies['Medal']

In [21]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9218859667230915

Swimming Logistics Regression

In [22]:
# Creating on the basis of Athlethics
Swimming_df = data_df[data_df['Sport'] == "Swimming"]
Swimming_df = Swimming_df.dropna()
  
print(Swimming_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
64      NLD  Swimming  False     0           1              0             2   
65      NLD  Swimming  False     0           1              0             2   
66      NLD  Swimming  False     0           1              0             2   
67      NLD  Swimming  False     0           1              0             2   
68      NLD  Swimming  False     0           1              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203887  RUS  Swimming  False     0           0              0             2   
203888  RUS  Swimming  False     0           0              0             2   
203942  NLD  Swimming  False     0           1              0             2   
203943  NLD  Swimming  False     0           1              0             2   
203944  NLD  Swimming   True     0           1              0             1   

       Life_Expect_Bin Global_GDP_Perc_Bin  
64    

In [23]:
Swimming_dummies = pd.get_dummies(Swimming_df)
Swimming_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
64,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
65,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
66,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [24]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Swimming_dummies.drop(columns=['Medal'])
y_train = Swimming_dummies['Medal']

In [25]:
x_test = Swimming_dummies.drop(columns=['Medal'])
y_test = Swimming_dummies['Medal']

In [26]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9132554711418528

Gymnastics Logistic Regression

In [27]:
Gymnastics_df = data_df[data_df['Sport'] == "Gymnastics"]
  
print(Gymnastics_df)

        NOC       Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
85      ESP  Gymnastics  False     0           1              0             1   
86      ESP  Gymnastics  False     0           1              0             1   
87      ESP  Gymnastics  False     0           1              0             1   
88      ESP  Gymnastics  False     0           1              0             1   
89      ESP  Gymnastics  False     0           1              0             1   
...     ...         ...    ...   ...         ...            ...           ...   
203699  GDR  Gymnastics   True     0         NaN              0           NaN   
203700  GDR  Gymnastics  False     0         NaN              0           NaN   
203701  GDR  Gymnastics   True     0         NaN              0           NaN   
203702  GDR  Gymnastics   True     0         NaN              0           NaN   
203703  GDR  Gymnastics  False     0         NaN              0           NaN   

       Life_Expect_Bin Glob

In [28]:
Gymnastics_df = Gymnastics_df.dropna()
Gymnastics_df


,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
85,ESP,Gymnastics,False,0,1,0,1,8,0
86,ESP,Gymnastics,False,0,1,0,1,8,0
87,ESP,Gymnastics,False,0,1,0,1,8,0
88,ESP,Gymnastics,False,0,1,0,1,8,0
89,ESP,Gymnastics,False,0,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
203680,ITA,Gymnastics,False,0,0,0,3,7,0
203681,ITA,Gymnastics,False,0,0,0,3,7,0
203682,ITA,Gymnastics,False,0,0,0,3,7,0
203683,ITA,Gymnastics,False,0,0,0,3,7,0


In [29]:
Gymnastics_dummies = pd.get_dummies(Gymnastics_df)
Gymnastics_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGD,NOC_BGR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
85,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
86,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
87,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [30]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Gymnastics_dummies.drop(columns=['Medal'])
y_train = Gymnastics_dummies['Medal']

In [31]:
x_test = Gymnastics_dummies.drop(columns=['Medal'])
y_test = Gymnastics_dummies['Medal']

In [32]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9492940640655837

Cycling Logistic Regression

In [33]:
Cycling_df = data_df[data_df['Sport'] == "Cycling"]
  
print(Cycling_df)


        NOC    Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
131     RUS  Cycling   True     0           0              0             2   
132     RUS  Cycling  False     0           0              0             2   
173     EGY  Cycling  False     0           0              0             5   
204     USA  Cycling  False     0           2              1             1   
241     EGY  Cycling  False     0           0              0             5   
...     ...      ...    ...   ...         ...            ...           ...   
203744  CHE  Cycling  False     0           2              0             2   
203745  CHE  Cycling  False     0           1              0             2   
203746  CHE  Cycling  False     0           1              0             2   
203776  MNG  Cycling  False     0           0              0             4   
204018  CZE  Cycling  False     0         NaN              0             3   

       Life_Expect_Bin Global_GDP_Perc_Bin  
131               

In [34]:
Cycling_df = Cycling_df.dropna()
Cycling_df

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
131,RUS,Cycling,True,0,0,0,2,6,0
132,RUS,Cycling,False,0,0,0,2,6,0
173,EGY,Cycling,False,0,0,0,5,7,0
204,USA,Cycling,False,0,2,1,1,8,5
241,EGY,Cycling,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203743,CHE,Cycling,False,0,2,0,2,8,0
203744,CHE,Cycling,False,0,2,0,2,8,0
203745,CHE,Cycling,False,0,1,0,2,8,0
203746,CHE,Cycling,False,0,1,0,2,8,0


In [35]:
Cycling_dummies = pd.get_dummies(Cycling_df)
Cycling_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
131,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
132,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
173,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [36]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Cycling_dummies.drop(columns=['Medal'])
y_train = Cycling_dummies['Medal']

In [37]:
x_test = Cycling_dummies.drop(columns=['Medal'])
y_test = Cycling_dummies['Medal']

In [38]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9057317427989525

In [39]:
#Get Sport count to use for later dataframe creation
Sport_counts = data_df["Sport"].value_counts()
Sport_counts

Athletics                27915
Swimming                 19902
Gymnastics               18095
Cycling                   7886
Rowing                    7800
Shooting                  7274
Fencing                   6309
Canoeing                  5529
Wrestling                 5170
Football                  4831
Boxing                    4648
Equestrianism             4474
Judo                      3774
Basketball                3563
Handball                  3535
Volleyball                3384
Weightlifting             3145
Water Polo                2746
Archery                   2057
Diving                    2051
Tennis                    2016
Table Tennis              1937
Badminton                 1455
Modern Pentathlon         1254
Synchronized Swimming      895
Baseball                   894
Rhythmic Gymnastics        654
Taekwondo                  606
Beach Volleyball           564
Triathlon                  529
Softball                   478
Rugby Sevens               299
Trampoli

Rowing Logistic Regression

In [40]:
Rowing_df = data_df[data_df['Sport'] == "Rowing"]
Rowing_df = Rowing_df.dropna()
 
print(Rowing_df)

        NOC   Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
60      NLD  Rowing   True     0           1              0             2   
61      NLD  Rowing  False     0           1              0             2   
98      ITA  Rowing   True     0           1              0             2   
116     ARG  Rowing  False     0           0              0             0   
130     CHL  Rowing  False     0           0              0             1   
...     ...     ...    ...   ...         ...            ...           ...   
203971  NLD  Rowing  False     0           0              0             2   
203972  NLD  Rowing   True     0           1              0             2   
203973  NLD  Rowing   True     0           1              0             2   
204025  RUS  Rowing  False     0           0              0             2   
204026  RUS  Rowing  False     0           0              0             2   

       Life_Expect_Bin Global_GDP_Perc_Bin  
60                   8        

In [41]:
Rowing_dummies = pd.get_dummies(Rowing_df)
Rowing_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
60,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
61,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
98,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [42]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Rowing_dummies.drop(columns=['Medal'])
y_train = Rowing_dummies['Medal']

x_test = Rowing_dummies.drop(columns=['Medal'])
y_test = Rowing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7811712846347607

Shooting Logistic Regression

In [43]:
Shooting_df = data_df[data_df['Sport'] == "Shooting"]
Shooting_df = Shooting_df.dropna()
 
print(Shooting_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
79      NOR  Shooting  False     0           0              0             4   
124     CHL  Shooting  False     0           0              0             1   
137     CHL  Shooting  False     0           0              0             1   
237     EGY  Shooting  False     0           0              0             5   
282     EGY  Shooting  False     0           0              0             5   
...     ...       ...    ...   ...         ...            ...           ...   
203765  CHE  Shooting  False     0           2              0             2   
203766  CHE  Shooting  False     0           2              0             2   
203767  CHE  Shooting  False     0           2              0             2   
203948  NOR  Shooting  False     0           0              0             4   
203949  NOR  Shooting  False     0           0              0             4   

       Life_Expect_Bin Global_GDP_Perc_Bin  
79    

In [44]:
Shooting_dummies = pd.get_dummies(Shooting_df)
Shooting_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
79,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
124,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
137,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [45]:
x_train = Shooting_dummies.drop(columns=['Medal'])
y_train = Shooting_dummies['Medal']

x_test = Shooting_dummies.drop(columns=['Medal'])
y_test = Shooting_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9352194180999839

Fencing Logistic Regression

In [46]:
Fencing_df = data_df[data_df['Sport'] == "Fencing"]
Fencing_df = Fencing_df.dropna()
 
print(Fencing_df)

        NOC    Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
99      ESP  Fencing   True     0           1              0             2   
117     GRC  Fencing  False     0           1              0             2   
141     MEX  Fencing  False     1           0              0             3   
142     MEX  Fencing  False     1           0              0             3   
143     MEX  Fencing  False     1           0              0             3   
...     ...      ...    ...   ...         ...            ...           ...   
203312  AUT  Fencing  False     0           0              0             3   
203313  AUT  Fencing  False     0           0              0             3   
203408  CHN  Fencing  False     0           0              8             7   
203409  CHN  Fencing  False     0           0              8             7   
203962  POL  Fencing  False     0           0              0             3   

       Life_Expect_Bin Global_GDP_Perc_Bin  
99                

In [47]:
Fencing_dummies = pd.get_dummies(Fencing_df)
Fencing_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BEN,NOC_BFA,NOC_BGR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
99,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
117,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
141,False,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [48]:
x_train = Fencing_dummies.drop(columns=['Medal'])
y_train = Fencing_dummies['Medal']

x_test = Fencing_dummies.drop(columns=['Medal'])
y_test = Fencing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8693467336683417

Canoeing Logistic Regression

In [49]:
Canoeing_df = data_df[data_df['Sport'] == "Canoeing"]
Canoeing_df = Canoeing_df.dropna()
 
print(Canoeing_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
109     CUB  Canoeing  False     0           0              0             2   
110     CUB  Canoeing  False     0           0              0             2   
111     BLR  Canoeing  False     0           0              0             2   
112     BLR  Canoeing   True     0           0              0             2   
203     CAN  Canoeing  False     1           0              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203817  USA  Canoeing  False     0           1              1             1   
203818  USA  Canoeing  False     0           1              1             1   
203819  USA  Canoeing  False     0           2              1             1   
203820  USA  Canoeing  False     0           2              1             1   
203861  LTU  Canoeing  False     0           0              0             3   

       Life_Expect_Bin Global_GDP_Perc_Bin  
109   

In [50]:
Canoeing_dummies = pd.get_dummies(Canoeing_df)
Canoeing_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
109,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
110,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
111,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [51]:
x_train = Canoeing_dummies.drop(columns=['Medal'])
y_train = Canoeing_dummies['Medal']

x_test = Canoeing_dummies.drop(columns=['Medal'])
y_test = Canoeing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8574193548387097

Wrestling Logistic Regression

In [52]:
Wrestling_df = data_df[data_df['Sport'] == "Wrestling"]
Wrestling_df = Wrestling_df.dropna()
 
print(Wrestling_df)

        NOC      Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
52      NOR  Wrestling  False     0           1              0             2   
53      NOR  Wrestling  False     0           2              0             2   
84      EST  Wrestling  False     0           0              0             2   
125     USA  Wrestling   True     0           1              1             1   
207     EGY  Wrestling  False     0           0              0             5   
...     ...        ...    ...   ...         ...            ...           ...   
203710  HRV  Wrestling  False     0           0              0             3   
203711  HRV  Wrestling  False     0           0              0             3   
203779  USA  Wrestling  False     1           1              1             2   
203924  LVA  Wrestling  False     0           0              0             3   
204017  GRC  Wrestling  False     0           1              0             2   

       Life_Expect_Bin Global_GDP_Perc_

In [53]:
Wrestling_dummies = pd.get_dummies(Wrestling_df)
Wrestling_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_ALB,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
52,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
53,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
84,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [54]:
x_train = Wrestling_dummies.drop(columns=['Medal'])
y_train = Wrestling_dummies['Medal']

x_test = Wrestling_dummies.drop(columns=['Medal'])
y_test = Wrestling_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.861604997597309

Football Logistic Regression

In [55]:
Football_df = data_df[data_df['Sport'] == "Football"]
Football_df = Football_df.dropna()
 
print(Football_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
69      NOR  Football   True     0           1              0             2   
118     CMR  Football   True     0           0              0             5   
122     GRC  Football  False     0           1              0             2   
134     ITA  Football  False     0           1              0             2   
166     IRQ  Football  False     0           0              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203675  SAU  Football  False     0           0              0             1   
203725  NLD  Football  False     0           2              0             1   
203784  ESP  Football  False     0           0              0             2   
203859  CAN  Football  False     0           2              0             1   
204024  USA  Football  False     0           0              2             2   

       Life_Expect_Bin Global_GDP_Perc_Bin  
69    

In [56]:
Football_dummies = pd.get_dummies(Football_df)
Football_dummies.head(3)

,Medal,Host,NOC_ARE,NOC_ARG,NOC_AUS,NOC_BEL,NOC_BLR,NOC_BRA,NOC_CAN,NOC_CHE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
69,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
118,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
122,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [57]:
x_train = Football_dummies.drop(columns=['Medal'])
y_train = Football_dummies['Medal']

x_test = Football_dummies.drop(columns=['Medal'])
y_test = Football_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8740793537657401

Boxing Logistic Regression

In [58]:
Boxing_df = data_df[data_df['Sport'] == "Boxing"]
boxing_df = Boxing_df.dropna()
 
print(Boxing_df)

        NOC   Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
80      MAR  Boxing  False     0           0              0             3   
94      BGR  Boxing  False     0           0              0             3   
102     SDN  Boxing  False     0           0              0             7   
119     CMR  Boxing  False     0           0              0             7   
145     NGA  Boxing  False     0           0              0             7   
...     ...     ...    ...   ...         ...            ...           ...   
203780  CHL  Boxing  False     0           0              0             2   
203788  COL  Boxing  False     0           0              0             3   
203857  ITA  Boxing  False     0           0              0             3   
203877  BLR  Boxing   True     0           0              0             2   
203878  BLR  Boxing  False     0           0              0             2   

       Life_Expect_Bin Global_GDP_Perc_Bin  
80                   7        

In [59]:
Boxing_dummies = pd.get_dummies(Boxing_df)
Boxing_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AFG,NOC_AGO,NOC_AND,NOC_ARG,NOC_ARM,NOC_ASM,NOC_ATG,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
80,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
94,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
102,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [60]:
x_train = Boxing_dummies.drop(columns=['Medal'])
y_train = Boxing_dummies['Medal']

x_test = Boxing_dummies.drop(columns=['Medal'])
y_test = Boxing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8597246127366609

Equestrianism Logistic Regression

In [61]:
Equestrianism_df = data_df[data_df['Sport'] == "Equestrianism"]
Equestrianism_df = Boxing_df.dropna()
 
print(Equestrianism_df)

        NOC   Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
80      MAR  Boxing  False     0           0              0             3   
94      BGR  Boxing  False     0           0              0             3   
102     SDN  Boxing  False     0           0              0             7   
119     CMR  Boxing  False     0           0              0             7   
145     NGA  Boxing  False     0           0              0             7   
...     ...     ...    ...   ...         ...            ...           ...   
203780  CHL  Boxing  False     0           0              0             2   
203788  COL  Boxing  False     0           0              0             3   
203857  ITA  Boxing  False     0           0              0             3   
203877  BLR  Boxing   True     0           0              0             2   
203878  BLR  Boxing  False     0           0              0             2   

       Life_Expect_Bin Global_GDP_Perc_Bin  
80                   7        

In [62]:
Equestrianism_dummies = pd.get_dummies(Equestrianism_df)
Equestrianism_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_AGO,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
80,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
94,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
102,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [63]:
x_train = Equestrianism_dummies.drop(columns=['Medal'])
y_train = Equestrianism_dummies['Medal']

x_test = Equestrianism_dummies.drop(columns=['Medal'])
y_test = Equestrianism_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8773800456968773

Judo Logistic Regression

In [64]:
Judo_df = data_df[data_df['Sport'] == "Judo"]
Judo_df = Judo_df.dropna()
 
print(Judo_df)

        NOC Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
1       CHN  Judo  False     0           0              7             4   
100     TCD  Judo  False     0           0              0             7   
120     TUR  Judo  False     0           0              0             3   
300     EGY  Judo  False     0           0              0             5   
314     EGY  Judo  False     0           0              0             5   
...     ...   ...    ...   ...         ...            ...           ...   
203810  CAN  Judo  False     0           2              0             1   
203811  CAN  Judo  False     0           1              0             1   
203922  GEO  Judo   True     0           0              0             4   
203960  NLD  Judo   True     0           1              0             2   
203961  NLD  Judo  False     0           1              0             1   

       Life_Expect_Bin Global_GDP_Perc_Bin  
1                    7                   2  
100      

In [65]:
Judo_dummies = pd.get_dummies(Judo_df)
Judo_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AFG,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
1,False,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
100,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
120,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [66]:
x_train = Judo_dummies.drop(columns=['Medal'])
y_train = Judo_dummies['Medal']

x_test = Judo_dummies.drop(columns=['Medal'])
y_test = Judo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8726263511539585

Basketball Logistic Regression

In [67]:
Basketball_df = data_df[data_df['Sport'] == "Basketball"]
Basketball_df = Basketball_df.dropna()
 
print(Basketball_df)

        NOC       Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
0       CHN  Basketball  False     0           0              8             6   
107     ESP  Basketball  False     0           1              0             2   
192     ITA  Basketball  False     0           0              0             3   
257     EGY  Basketball  False     0           0              0             5   
264     EGY  Basketball  False     0           0              0             5   
...     ...         ...    ...   ...         ...            ...           ...   
203733  LTU  Basketball  False     0           0              0             3   
203734  LTU  Basketball   True     0           0              0             3   
203735  LTU  Basketball  False     0           0              0             3   
203956  DEU  Basketball  False     0           2              0             2   
204027  BLR  Basketball  False     0           0              0             2   

       Life_Expect_Bin Glob

In [68]:
Basketball_dummies = pd.get_dummies(Basketball_df)
Basketball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_BGR,NOC_BLR,NOC_BRA,NOC_CAF,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
0,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
107,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
192,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [69]:
x_train = Basketball_dummies.drop(columns=['Medal'])
y_train = Basketball_dummies['Medal']

x_test = Basketball_dummies.drop(columns=['Medal'])
y_test = Basketball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8938140919616275

Handball Logistic Regression

In [70]:
Handball_df = data_df[data_df['Sport'] == "Handball"]
Handball_df = Handball_df.dropna()
 
print(Handball_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
50      NOR  Handball   True     0           4              0             2   
113     FRA  Handball   True     0           2              0             2   
114     FRA  Handball   True     0           1              0             1   
115     FRA  Handball   True     0           1              0             1   
135     FRA  Handball  False     0           1              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203758  CUB  Handball  False     0           0              0             2   
203770  CHE  Handball  False     0           2              0             2   
203787  ESP  Handball  False     0           0              0             2   
203858  AUT  Handball  False     0           1              0             3   
203925  SVN  Handball  False     0           0              0             4   

       Life_Expect_Bin Global_GDP_Perc_Bin  
50    

In [71]:
Handball_dummies = pd.get_dummies(Handball_df)
Handball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BRA,NOC_CAN,NOC_CHE,NOC_CHN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
50,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
113,True,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
114,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [72]:
x_train = Handball_dummies.drop(columns=['Medal'])
y_train = Handball_dummies['Medal']

x_test = Handball_dummies.drop(columns=['Medal'])
y_test = Handball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.7936344969199178

Volleyball

In [73]:
Volleyball_df = data_df[data_df['Sport'] == "Volleyball"]
Volleyball_df = Volleyball_df.dropna()
 
print(Volleyball_df)

        NOC       Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
210     EGY  Volleyball  False     0           0              0             5   
218     EGY  Volleyball  False     0           0              0             5   
236     EGY  Volleyball  False     0           0              0             5   
254     EGY  Volleyball  False     0           0              0             5   
270     EGY  Volleyball  False     0           0              0             5   
...     ...         ...    ...   ...         ...            ...           ...   
203795  CHN  Volleyball  False     0           0              8             7   
203945  NLD  Volleyball  False     0           0              0             2   
203946  NLD  Volleyball   True     0           1              0             2   
203947  NLD  Volleyball   True     0           1              0             2   
204009  POL  Volleyball  False     0           0              0             3   

       Life_Expect_Bin Glob

In [74]:
Volleyball_dummies = pd.get_dummies(Volleyball_df)
Volleyball_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,NOC_CHN,NOC_CMR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
210,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
218,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
236,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [75]:
x_train = Volleyball_dummies.drop(columns=['Medal'])
y_train = Volleyball_dummies['Medal']

x_test = Volleyball_dummies.drop(columns=['Medal'])
y_test = Volleyball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8166845685642679

Weightlifting

In [76]:
Weightlifting_df = data_df[data_df['Sport'] == "Weightlifting"]
Weightlifting_df = Weightlifting_df.dropna()
 
print(Weightlifting_df)

        NOC          Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin  \
51      ROU  Weightlifting  False     0           0              0   
165     IRQ  Weightlifting  False     0           0              0   
176     QAT  Weightlifting  False     0           1              0   
188     AZE  Weightlifting  False     0           0              0   
195     IRQ  Weightlifting  False     0           0              0   
...     ...            ...    ...   ...         ...            ...   
203502  MAR  Weightlifting  False     0           0              0   
203658  CZE  Weightlifting  False     0           0              0   
203855  ESP  Weightlifting  False     0           0              0   
203856  ESP  Weightlifting  False     1           0              0   
203937  POL  Weightlifting  False     0           0              0   

       Rural_Pop_Bin Life_Expect_Bin Global_GDP_Perc_Bin  
51                 4               7                   0  
165                4               5     

In [77]:
Weightlifting_dummies = pd.get_dummies(Weightlifting_df)
Weightlifting_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
51,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
165,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
176,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [78]:
x_train = Weightlifting_dummies.drop(columns=['Medal'])
y_train = Weightlifting_dummies['Medal']

x_test = Weightlifting_dummies.drop(columns=['Medal'])
y_test = Weightlifting_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8793300342596118

Water Polo

In [79]:
Polo_df = data_df[data_df['Sport'] == "Water Polo"]
Polo_df = Polo_df.dropna()
 
print(Polo_df)

        NOC       Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
58      NLD  Water Polo  False     0           0              0             3   
121     ESP  Water Polo   True     0           0              0             2   
189     ITA  Water Polo  False     0           1              0             2   
251     EGY  Water Polo  False     0           0              0             5   
267     EGY  Water Polo  False     0           0              0             5   
...     ...         ...    ...   ...         ...            ...           ...   
203275  RUS  Water Polo  False     0           0              0             2   
203613  ESP  Water Polo  False     0           0              0             3   
203630  RUS  Water Polo  False     0           0              0             2   
203631  KAZ  Water Polo  False     0           0              0             4   
203719  NLD  Water Polo  False     0           1              0             2   

       Life_Expect_Bin Glob

In [80]:
Polo_dummies = pd.get_dummies(Polo_df)
Polo_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,NOC_CHN,NOC_CUB,NOC_CZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
58,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
121,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
189,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [81]:
x_train = Polo_dummies.drop(columns=['Medal'])
y_train = Polo_dummies['Medal']

x_test = Polo_dummies.drop(columns=['Medal'])
y_test = Polo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8627450980392157

Archery

In [82]:
Archery_df = data_df[data_df['Sport'] == "Archery"]
Archery_df = Archery_df.dropna()
 
print(Archery_df)

        NOC    Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
206     MAR  Archery  False     0           0              0             4   
521     EGY  Archery  False     0           0              0             5   
764     RUS  Archery  False     0           0              0             2   
765     RUS  Archery  False     0           0              0             2   
845     IRN  Archery  False     0           0              0             2   
...     ...      ...    ...   ...         ...            ...           ...   
202043  CHN  Archery  False     0           0              8             5   
202044  CHN  Archery   True     0           0              8             5   
202045  CHN  Archery   True     1           0              7             4   
202046  CHN  Archery   True     1           0              7             4   
202286  CHN  Archery  False     0           0              8             7   

       Life_Expect_Bin Global_GDP_Perc_Bin  
206               

In [83]:
Archery_dummies = pd.get_dummies(Archery_df)
Archery_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGD,NOC_BGR,NOC_BLR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
206,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
521,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
764,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [84]:
x_train = Archery_dummies.drop(columns=['Medal'])
y_train = Archery_dummies['Medal']

x_test = Archery_dummies.drop(columns=['Medal'])
y_test = Archery_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9129082426127527

Diving

In [85]:
Diving_df = data_df[data_df['Sport'] == "Diving"]
Diving_df = Diving_df.dropna()
 
print(Diving_df)

        NOC   Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
108     CUB  Diving  False     0           0              0             2   
235     EGY  Diving  False     0           0              0             5   
538     CAN  Diving  False     0           2              0             1   
539     CAN  Diving  False     0           2              0             1   
540     CAN  Diving   True     0           2              0             1   
...     ...     ...    ...   ...         ...            ...           ...   
202802  UKR  Diving  False     0           0              0             3   
202803  UKR  Diving  False     0           0              0             3   
202804  UKR  Diving  False     0           0              0             3   
202805  UKR  Diving   True     0           0              0             3   
202806  UKR  Diving  False     0           0              0             3   

       Life_Expect_Bin Global_GDP_Perc_Bin  
108                  8        

In [86]:
Diving_dummies = pd.get_dummies(Diving_df)
Diving_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
108,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
235,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
538,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [87]:
x_train = Diving_dummies.drop(columns=['Medal'])
y_train = Diving_dummies['Medal']

x_test = Diving_dummies.drop(columns=['Medal'])
y_test = Diving_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8816234498308907

Tennis

In [88]:
Tennis_df = data_df[data_df['Sport'] == "Tennis"]
Tennis_df = Tennis_df.dropna()
 
print(Tennis_df)

        NOC   Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
123     MEX  Tennis  False     0           0              0             2   
446     NGA  Tennis  False     0           0              0             6   
1008    BRA  Tennis  False     0           0              1             2   
1178    ZAF  Tennis  False     0           0              0             4   
1607    USA  Tennis   True     1           1              1             2   
...     ...     ...    ...   ...         ...            ...           ...   
203920  BLR  Tennis  False     0           0              0             2   
203921  BLR  Tennis  False     0           0              0             2   
203926  RUS  Tennis   True     0           0              0             2   
203927  RUS  Tennis  False     0           0              0             2   
203928  RUS  Tennis  False     0           0              0             2   

       Life_Expect_Bin Global_GDP_Perc_Bin  
123                  7        

In [89]:
Tennis_dummies = pd.get_dummies(Tennis_df)
Tennis_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BEN,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
123,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
446,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1008,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [90]:
x_train = Tennis_dummies.drop(columns=['Medal'])
y_train = Tennis_dummies['Medal']

x_test = Tennis_dummies.drop(columns=['Medal'])
y_test = Tennis_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9209989806320081

Table Tennis

In [91]:
Table_Tennis_df = data_df[data_df['Sport'] == "Table Tennis"]
Table_Tennis_df = Table_Tennis_df.dropna()
 
print(Table_Tennis_df)

        NOC         Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin  \
369     EGY  Table Tennis  False     0           0              0   
370     EGY  Table Tennis  False     0           0              0   
371     EGY  Table Tennis  False     0           0              0   
611     NGA  Table Tennis  False     0           0              1   
962     IND  Table Tennis  False     0           0              7   
...     ...           ...    ...   ...         ...            ...   
202757  USA  Table Tennis  False     1           1              1   
202758  USA  Table Tennis  False     0           1              1   
202759  USA  Table Tennis  False     0           1              1   
202760  USA  Table Tennis  False     0           2              1   
203951  HUN  Table Tennis  False     0           0              0   

       Rural_Pop_Bin Life_Expect_Bin Global_GDP_Perc_Bin  
369                5               7                   0  
370                5               7                 

In [92]:
Table_Tennis_dummies = pd.get_dummies(Table_Tennis_df)
Table_Tennis_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BIH,NOC_BLR,NOC_BRA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
369,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
370,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
371,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [93]:
x_train = Table_Tennis_dummies.drop(columns=['Medal'])
y_train = Table_Tennis_dummies['Medal']

x_test = Table_Tennis_dummies.drop(columns=['Medal'])
y_test = Table_Tennis_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9215572341661824

Badminton

In [94]:
Badminton_df = data_df[data_df['Sport'] == "Badminton"]
Badminton_df = Badminton_df.dropna()
 
print(Badminton_df)

        NOC      Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
26      FIN  Badminton  False     0           1              0             1   
593     ESP  Badminton  False     0           1              0             2   
594     ESP  Badminton  False     0           1              0             1   
595     ESP  Badminton  False     0           1              0             1   
1239    GBR  Badminton  False     1           1              0             1   
...     ...        ...    ...   ...         ...            ...           ...   
203867  RUS  Badminton  False     0           0              1             2   
203868  RUS  Badminton  False     0           0              0             2   
203952  DEU  Badminton  False     0           2              0             2   
203953  DEU  Badminton  False     0           2              0             2   
203954  DEU  Badminton  False     0           1              0             2   

       Life_Expect_Bin Global_GDP_Perc_

In [95]:
Badminton_dummies = pd.get_dummies(Badminton_df)
Badminton_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BLR,NOC_BRA,NOC_BRN,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
26,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
593,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
594,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [96]:
x_train = Badminton_dummies.drop(columns=['Medal'])
y_train = Badminton_dummies['Medal']

x_test = Badminton_dummies.drop(columns=['Medal'])
y_test = Badminton_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8830242510699001

Modern Pentathlon

Synchronized Swimming

Baseball

Rhythmic Gymnastics

Taekwondo

Beach Volleyball

Triathlon

Softball

Rugby Sevens

Trampolining

Golf